In [59]:
import Base: iterate,max, exp, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero,isless
using BenchmarkTools
import Test: @test, @testset

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


In [58]:
2^2

4

# Define dual number

In [3]:
struct Dual{T <: Number} <: Number
    x::T
    dx::T
end
Dual(n::Integer, d::Float64) = Dual(promote(n, d)...)
Dual(n::Float64, d::Integer) = Dual(promote(n, d)...)

Dual

In [4]:
function convert(::Type{Dual{T}}, x::T) where {T}
   Dual(x, zero(x)) 
end
function convert(::Type{Dual{T}}, x::Dual{S}) where {S, T}
    Dual(T(x.x), T(x.dx))
end
# This is needed according to an error before
function convert(::Type{Dual{T}}, x::T) where {T <: Number}
    Dual(x, zero(x))
end
function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Dual(x_as_T, zero(x_as_T))
end

convert (generic function with 187 methods)

In [5]:
function zero(x::Dual{T}) where T
    Dual(zero(x.x), zero(x.dx))
end

zero (generic function with 23 methods)

In [6]:
function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S,T)}
end

promote_rule (generic function with 126 methods)

In [7]:
function extract_derivative(xdx::Dual)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

# Differential operator

In [8]:
function derivativeСalculation(f, value)
    function df(x)
        xdx = Dual(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df(value)
end
function derivativeСalculation(f,i::Integer,values)
    function df(valuesS)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(valuesS)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end
function derivativeСalculation(f,values)
    function df(xs...)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end

derivativeСalculation (generic function with 2 methods)

In [61]:
function +(x::Dual, y::Dual)
    Dual(x.x+y.x, x.dx+y.dx)
end
function -(x::Dual, y::Dual)
    Dual(x.x-y.x, x.dx-y.dx)
end
function -(x::Dual)
    Dual(-x.x, -x.dx)
end
function *(x::Dual, y::Dual)
#      println("mnożenie")
     Dual(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Dual, y::Dual)
    Dual(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end
function /(x::Array, y::Number)
    return [i / y for i in x]
end

function ^(a::Dual, x::Dual)
#     println("Daszek")
    Dual(a.x^x.x, a.dx * x.x * a.x ^ (x.x - 1) + x.dx * a.x ^ x.x * log(a.x)) 
end

function ^(a::Dual, x::Integer)
#     println("Daszek")
    Dual(a.x^x, a.dx * x * a.x ^ (x - 1)) 
end

^ (generic function with 68 methods)

In [10]:
function sqrt(x::Dual)
    Dual(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 21 methods)

In [11]:
function exp(x::Dual)
    return Dual(exp(x.x), exp(x.x)*x.dx)
end
function exp(xs::Array)
    print(xs)
    return [exp(x) for x in xs]
end
function sin(x::Dual)
    return Dual(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Dual)
    return Dual(cos(x.x), -sin(x.x)*x.dx)
end

function tan(x::Dual)
    return Dual(tan(x.x),(1/cos(x.x)^2)*x.dx)
end
isless(x::Dual, y::Dual) = x.x < y.x;

In [12]:
function max(x::Dual)
    return Dual(max(0,x.x), x.x < 0 ? 0 : 1 * x.dx)
end

max (generic function with 13 methods)

In [13]:
function softmax(vector::Array)
    e = exp(vector)
    return e / sum(e)
#     z = [Dual(2.0, 1.0), Dual(21.0, 1.0)]
#     s = zero(vector[1])
#     for x in z
#        s = s + x 
#     end    
#     return e / s
end

softmax (generic function with 1 method)

In [14]:
function iterate(iter::Vector{Dual}, state=1)
#     println(iter, "Test22232323")
    if state > length(iter)
        return nothing
    end
    return (iter[state],state+1)
end

iterate (generic function with 212 methods)

In [15]:
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        temp  = [f(x)...]
        column = extract_derivative.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [16]:
f(x::Vector) = softmax(x)
J(softmax, [1., 2.])

Dual{Float64}[Dual{Float64}(1.0, 1.0), Dual{Float64}(2.0, 0.0)]Dual{Float64}[Dual{Float64}(1.0, 1.0), Dual{Float64}(2.0, 0.0)]Dual{Float64}[Dual{Float64}(1.0, 0.0), Dual{Float64}(2.0, 1.0)]Dual{Float64}[Dual{Float64}(1.0, 0.0), Dual{Float64}(2.0, 1.0)]

2×2 Array{Float64,2}:
  0.196612  -0.196612
 -0.196612   0.196612

In [17]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x)]
J(f, [1, 2., 3])

5×3 Array{Float64,2}:
 1.0        0.0         0.0
 0.0        0.0        -0.555556
 0.0       16.0        -2.0
 1.62091    0.0         0.841471
 0.377539  -0.0755078  -0.0755078

In [18]:
function linear(x)
    return x^2
end


linear (generic function with 1 method)

In [19]:
derivativeСalculation(linear,1,[12])

24

In [20]:
softmax([1,2,3])

[1, 2, 3]

3-element Array{Float64,1}:
 0.09003057317038046
 0.24472847105479767
 0.6652409557748219

In [21]:
dldx(1.2,2,3)

LoadError: [91mUndefVarError: dldx not defined[39m

In [22]:
function relu(x)
    return max(0,x)
end

relu (generic function with 1 method)

In [23]:
function f(x)
    return x+relu(x*x)
end

f (generic function with 2 methods)

In [24]:
d=D(ftest)

LoadError: [91mUndefVarError: D not defined[39m

In [25]:
d(2.0)

LoadError: [91mUndefVarError: d not defined[39m

In [26]:
Dual(2,4)+Dual(4.4, 2.1)

Dual{Float64}(6.4, 6.1)

In [65]:
function rosenbrock(x::Vector)
    value = zero(x[1])
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end


rosenbrock (generic function with 2 methods)

In [91]:
# d = derivativeСalculation(rosenbrock,[1.0,2.0,3.0])
@benchmark J(rosenbrock, [rand(1000,1)...])

BenchmarkTools.Trial: 
  memory estimate:  32.30 MiB
  allocs estimate:  21021
  --------------
  minimum time:     61.076 ms (0.00% GC)
  median time:      63.394 ms (2.19% GC)
  mean time:        63.776 ms (1.76% GC)
  maximum time:     68.505 ms (1.98% GC)
  --------------
  samples:          79
  evals/sample:     1

In [78]:
[rand(3,1)...]

0.5340167841882746

In [86]:
s = rand(3,1)
print(s[1][1])

0.223582100365062

In [92]:
J(rosenbrock, [1., 2., 3, 4, 5,6 ,7])

1×7 Array{Float64,2}:
 -400.0  1002.0  5804.0  16606.0  35808.0  65810.0  -5800.0

In [29]:
rosenbrock([1,2])

100

In [43]:
function rosenbrock(x)
    value = zero(x[1])
#     value = (1-a)^2 + 100*(x - a^2)^2
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^3
    end
    value
end

rosenbrock (generic function with 2 methods)

In [31]:
derivativeСalculation(rosenbrock,1,[1,2,3])

LoadError: [91mMethodError: no method matching rosenbrock(::Dual{Int64}, ::Int64, ::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  rosenbrock(::Any) at In[30]:1[39m

In [32]:
tan(2)

-2.185039863261519

In [93]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
@benchmark J(f,[rand(3,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  5.20 KiB
  allocs estimate:  77
  --------------
  minimum time:     3.273 μs (0.00% GC)
  median time:      3.661 μs (0.00% GC)
  mean time:        4.419 μs (8.19% GC)
  maximum time:     455.269 μs (98.17% GC)
  --------------
  samples:          10000
  evals/sample:     8

In [94]:
J(f, [2.9,2.2,1546.23])

6×3 Array{Float64,2}:
     1.0          0.0          0.0
     0.0          0.0         -2.09133e-6
     0.0         17.6         -2.0
 -1501.32         0.0          0.239249
     0.0117077   -7.55172e-6  -7.55172e-6
     5.8        -14.52         0.0